In [1]:
pip install psx-data-reader

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1050]:
from psx import stocks, tickers
import datetime

In [1051]:
tickers = tickers()

In [1052]:
today = datetime.date.today()
yesterday = yesterday = today - datetime.timedelta(days=1)


In [1053]:

import json


# Dictionary that map the ticket on a the company name
ticker_to_company = {
    "BOP":"Bank of Punjab",
    "PSO":"Pakistan State Oil",
    "AKBL":"Askari bank",
    }




company=None

def enter_company():
  # Get user input
  user_input = input("Enter the company name: ")

  # Search for the corresponding key
  found_ticker = None
  for ticker, company in ticker_to_company.items():
      if user_input.lower() == company.lower():
          found_ticker = ticker
          break

  if found_ticker is not None:
    # print(stock_name)
    company = found_ticker
    print(company)
    return company

  else:
    print("Company not found")




In [1054]:
# trained_model = {
#     "BOP":0,
#     "BATA":0,
#     "SNGPL":0,
#     "AVN":0,
#     "BWCL":0,
#     "PSO":0,
# }

# # Save the dictionary to a JSON file
# with open('trained_model.json', 'w') as json_file:
#     json.dump(trained_model, json_file)

In [1055]:
data = """ABL ABOT AGP AICL AKBL APL ARPL ATRL AVN BAFL BAHL BIPL BNWM BOP BWCL CEPB CHCC CNERGY COLG DAWH DCR DGKC EFERT EFUG ENGRO EPCL FABL FATIMA FCCL FCEPL FFBL FFC FHAM GADT GATM GHGL GLAXO HBL HGFA HINOON HMB HUBC IBFL ILP INDU INIL ISL JDWS JVDC KAPCO KEL KOHC KTML LCI LOTCHEM LUCK MARI MCB MEBL MLCF MTL MUGHAL MUREB NATF NBP NESTLE NML NRL OGDC PABC PAEL PAKT PGLC PIBTL PIOC PKGP PKGS POL POML PPL PSEL PSO PSX PTC RMPL SCBPL SEARL SHEL SHFA SNGP SPWL SRVI SYS TGL THALL TRG UBL UNITY UPFL YOUW"""
stock_name = enter_company()
while(stock_name is None):
  stock_name=enter_company()

Enter the company name: kjkf
Company not found
Enter the company name: bank of punjab
BOP


In [1056]:
# # Load the dictionary from the JSON file
# with open('trained_model.json', 'r') as json_file:
#     loaded_model = json.load(json_file)

# # Add a new key-value pair
# new_key = "FATIMA"
# new_value = 0
# loaded_model[new_key] = new_value

# # Delete a key-value pair
# key_to_delete = "NESTLE"
# if key_to_delete in loaded_model:
#     del loaded_model[key_to_delete]


# # Save the updated dictionary back to the file
# with open('trained_model.json', 'w') as json_file:
#     json.dump(loaded_model, json_file)

In [1057]:
print(stock_name)
data = stocks(stock_name, start=datetime.date(2020, 1, 1), end=yesterday)

df = data.reset_index()

BOP


In [1058]:

df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
name_csv = stock_name+".csv"
df.to_csv(name_csv, index=False)
df

,Date,Open,High,Low,Close,Volume
0,2020-01-01,11.37,11.87,11.34,11.73,7507000.0
1,2020-01-02,11.80,12.21,11.80,12.04,27923000.0
2,2020-01-03,12.08,12.33,11.80,11.85,11222000.0
3,2020-01-06,11.55,11.80,11.35,11.48,16874500.0
4,2020-01-07,11.65,11.80,11.45,11.74,4795000.0
...,...,...,...,...,...,...
981,2023-12-14,6.23,7.08,6.23,6.97,89773616.0
982,2023-12-15,7.19,7.65,7.04,7.52,112610953.0
983,2023-12-18,7.99,8.20,7.36,7.57,91711580.0
984,2023-12-19,7.55,7.55,6.57,6.72,118013188.0


In [1059]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from keras.models import load_model
from datetime import datetime

In [1060]:
from datetime import datetime, timedelta


### Preprocess

In [1061]:
# Load data
data = pd.read_csv(name_csv)
data2 = data
# Convert the 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Sort the data by date
data = data.sort_values(by='Date')

# Use only relevant columns
data = data[['Open', 'High', 'Low', 'Close', 'Volume']]

# Normalize the data using MinMaxScaler
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Define the number of time steps (you can experiment with this)
time_steps = 90


# Create sequences of data
X, y = [], []
for i in range(len(data_scaled) - time_steps):
    X.append(data_scaled[i:i+time_steps,:5])
    y.append(data_scaled[i+time_steps, 3])  # 'Close' column is at index 3

X, y = np.array(X), np.array(y)

print(len(X))
print(X.shape[0],X.shape[1],X.shape[2])

6
6 980 5


In [1062]:
def create_model():
  model = Sequential()
  model.add(LSTM(units=4, activation='relu', input_shape=(X.shape[1], X.shape[2])))
  model.add(Dense(units=1))
  model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])
  model.fit(X, y, epochs=5, batch_size=32)
  model.save(stock_name +'.h5')
  return model

In [1063]:
# Load the dictionary from the JSON file
with open('trained_model.json', 'r') as json_file:
  loaded_model = json.load(json_file)


if(loaded_model[stock_name]==1):
  model = load_model(stock_name +'.h5')
else:
  model = create_model()

  loaded_model[stock_name]=1 #Updating the value

  # Save the modified dictionary back to the file
  with open('trained_model.json', 'w') as json_file:
      json.dump(loaded_model, json_file)

In [1064]:
# timestamp = datetime.now().strftime("%Y%m%d%H%M%S")


In [1065]:
print (X.shape)

(6, 980, 5)


In [1066]:
# Evaluate the model on the test set
loss = model.evaluate(X, y)
print(f"Test Loss: {loss}")


1/1 [==============================] - 0s 292ms/step - loss: 0.0621 - accuracy: 0.0000e+00
Test Loss: [0.06213156506419182, 0.0]


In [1067]:
# Make predictions on the test set
predictions = model.predict(X)

# Inverse transform the predictions and actual values
predictions = scaler.inverse_transform(np.hstack((X[:, -1, :4], predictions.reshape(-1, 1))))
y_test_actual = scaler.inverse_transform(np.hstack((X[:, -1, :4], y.reshape(-1, 1))))

# Print dimensions of predictions and y_test_actual for debugging
print("Dimensions of predictions:", predictions.shape)
print("Dimensions of y_test_actual:", y_test_actual.shape)

# Calculate and print the root mean squared error (RMSE)
rmse = np.sqrt(np.mean((predictions[:, 3] - y_test_actual[:, 3])**2))
print(f"Root Mean Squared Error: {rmse}")

1/1 [==============================] - 0s 225ms/step
Dimensions of predictions: (6, 5)
Dimensions of y_test_actual: (6, 5)
Root Mean Squared Error: 0.0


In [1071]:

future_date = datetime.today().date()


# Use the latest available data to make predictions for the future date
latest_data = data2.iloc[-time_steps:]
latest_date = latest_data['Date'].min().date()


print(latest_date)
print(future_date)
count=0

while latest_date < future_date:

    input_sequence = latest_data[['Open', 'High', 'Low', 'Close', 'Volume']].values[-time_steps:]
    # print(latest_data.values)


    # print(input_sequence)

    input_sequence_scaled = input_sequence.reshape(1, time_steps, 5)

    # print(input_sequence_scaled)
    predicted_scaled = model.predict(input_sequence_scaled)

    print(predicted_scaled)
    # Inverse transform the predicted value to get the actual stock price
    predicted_price_scaled = np.array([[predicted_scaled[0, 0], 0, 0, 0, 0]])  # Add zeros for other columns
    predicted_price = scaler.inverse_transform(predicted_price_scaled)


    # Append the predicted price to the latest_data for the next iteration
    next_date = latest_data['Date'].max() + timedelta(days=1)
    latest_date = next_date

    latest_data = latest_data.append({'Date': next_date, 'Close': predicted_price[0][0]}, ignore_index=True)





print(latest_data)
print(f"Predicted Close Price for {future_date}: {latest_data.tail(1)['Close'].values[0]}")

2020-01-09
2023-12-21
1/1 [==============================] - 0s 58ms/step
[[0.00499283]]
          Date   Open   High    Low      Close       Volume
0   2020-01-09  12.19  12.73  12.19  12.570000   53074500.0
1   2020-01-10  12.69  13.40  12.53  13.330000   52475000.0
2   2020-01-13  13.60  13.72  13.13  13.280000   34974000.0
3   2020-01-14  13.25  13.65  13.25  13.380000   18020500.0
4   2020-01-15  13.40  13.59  13.15  13.500000   19650500.0
..         ...    ...    ...    ...        ...          ...
976 2023-12-15   7.19   7.65   7.04   7.520000  112610953.0
977 2023-12-18   7.99   8.20   7.36   7.570000   91711580.0
978 2023-12-19   7.55   7.55   6.57   6.720000  118013188.0
979 2023-12-20   6.55   6.90   5.81   6.540000   53704668.0
980 2023-12-21    NaN    NaN    NaN   3.344372          NaN

[981 rows x 6 columns]
Predicted Close Price for 2023-12-21: 3.3443718741647896


<ipython-input-1071-fffdb89ae1d1>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latest_data = latest_data.append({'Date': next_date, 'Close': predicted_price[0][0]}, ignore_index=True)
<ipython-input-1071-fffdb89ae1d1>:13: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  while latest_date < future_date:
